In [1]:
import pandas as pd
import numpy as np
import re
import recordlinkage
import pickle
import matplotlib.pyplot as plt
import Algorithmia

In [2]:
with open('../Data/all_muckrack_links.pkl', 'rb') as f:
    authors = pickle.load(f)
    
with open('../Data/news.pkl', 'rb') as f:
    news = pickle.load(f)
    
emails = pd.read_csv("../Data/compression_test.txt.gz", compression='gzip',sep = '\t')
emails = emails.fillna("")
emails = emails[['email_address', 'first_name_df', 'second_name_df']]

In [3]:
def first_name(url):
    return url[21:].split('-')[0]
first_name('https://muckrack.com/rafael-olmeda')

'rafael'

In [4]:
def author_outlet(author):
    return news[news.author == author].site_name.value_counts().index[0]

In [5]:
def concatenate_fields(author, outlet):
    return author + ' ' + outlet

In [6]:
authors['first_name_df'] = authors.apply(lambda x: first_name(x['request_url']), axis=1)

In [7]:
authors['common_outlet'] = authors.apply(lambda x: author_outlet(x['author']), axis=1)

In [8]:
authors['author + outlet'] = authors.apply(lambda x: concatenate_fields(x['author'], x['common_outlet']), axis=1)

In [9]:
authors

,author,article_count,request_url,redirect_url,method,confidence,first_name_df,common_outlet,author + outlet
0,Neil Shaw,523,https://muckrack.com/neil-shaw,https://muckrack.com/neil-shaw-1,brute force,-,neil,WalesOnline,Neil Shaw WalesOnline
1,James Rodger,512,https://muckrack.com/james-rodger,https://muckrack.com/james-rodger,brute force,-,james,BirminghamLive,James Rodger BirminghamLive
2,Jack Davis,251,https://muckrack.com/jack-davis,https://muckrack.com/jack-davis,brute force,-,jack,The Western Journal,Jack Davis The Western Journal
3,Adam Wells,223,https://muckrack.com/adam-wells,https://muckrack.com/adam-wells,brute force,-,adam,Bleacher Report,Adam Wells Bleacher Report
4,Sophie McCoid,218,https://muckrack.com/sophie-mccoid,https://muckrack.com/sophie-mccoid,brute force,-,sophie,Liverpool Echo,Sophie McCoid Liverpool Echo
...,...,...,...,...,...,...,...,...,...
3833,Levi Sumagaysay,10,https://muckrack.com/levi-sumagaysay,https://muckrack.com/levi-sumagaysay,brute force,-,levi,MarketWatch,Levi Sumagaysay MarketWatch
3834,Ben Ashford,10,https://muckrack.com/ben-ashford,https://muckrack.com/ben-ashford,brute force,-,ben,Mail Online,Ben Ashford Mail Online
3835,Craig Brown,10,https://muckrack.com/craig-brown,https://muckrack.com/craig-brown,brute force,-,craig,Mail Online,Craig Brown Mail Online
3836,Courtney Connley,10,https://muckrack.com/courtney-connley,https://muckrack.com/courtney-connley,brute force,-,courtney,CNBC,Courtney Connley CNBC


In [10]:
indexer = recordlinkage.Index()
indexer.block('first_name_df')
candidate_links = indexer.index(authors, emails)
print("candidate_links:", len(candidate_links))

candidate_links: 1296083


In [11]:
emails

,email_address,first_name_df,second_name_df
0,rating@mail.ru,,
1,xb3@mail.ru,,
2,info@aktiencheck.de,,
3,legals@heartmediact.com,,
4,8b4e078a51d04e0e9efdf470027f0ec1@sentry.wixpre...,,
...,...,...,...
2574181,hyzhenhua@mail.tzptt.zj.cn,,
2574182,hyzgkxni@gmail.com,,
2574183,hyzbhtbq@ygfqvw.com,,
2574184,hyzb@uibe.edu.cn,,


In [12]:
compare_cl = recordlinkage.Compare()
compare_cl.string('author + outlet', 'email_address', label = 'match')
features = compare_cl.compute(candidate_links, authors, emails)

In [13]:
features

match
0    7751     0.222222
     11897    0.222222
     28036    0.142857
     30146    0.142857
     58595    0.206897
...                ...
3833 2194924  0.161290
     2194925  0.111111
     2194926  0.222222
     2194927  0.222222
     2194942  0.148148

[1296083 rows x 1 columns]

In [14]:
author_indexes = set()
for i in features[features.match >= 0.5].index:
    author_indexes.add(i[0])

In [15]:
data = features[features.match >= 0.5].sort_index(level = [0, 1], sort_remaining = False)

In [16]:
data

match
1    2080352  0.551724
11   205311   0.500000
     314799   0.500000
33   38933    0.533333
     947087   0.533333
...                ...
3816 438843   0.600000
     1598269  0.560000
3830 687117   0.500000
3832 34788    0.512821
3833 152891   0.677419

[1788 rows x 1 columns]

In [17]:
data.loc[33].sort_values('match', ascending = False).head(1)

,match
38933,0.533333


In [18]:
matches = pd.DataFrame (columns = ['author_index', 'email_index', 'match_threshold', 'scraper_response'])

In [19]:
matches

,author_index,email_index,match_threshold,scraper_response


In [20]:
for i in author_indexes:
    temp = data.loc[i].sort_values('match', ascending = False).head(1)
    row = pd.Series(list([i, temp.index[0], round(temp.match.values[0], 3), '-']), index = matches.columns)
    matches = matches.append(row, ignore_index = True) 

In [21]:
matches

,author_index,email_index,match_threshold,scraper_response
0,1,2080352,0.552,-
1,2054,1764154,0.560,-
2,11,205311,0.500,-
3,2074,312243,0.615,-
4,33,38933,0.533,-
...,...,...,...,...
407,1989,6443,0.538,-
408,2011,785848,0.581,-
409,2022,809326,0.625,-
410,2023,5612,0.528,-


In [23]:
client = Algorithmia.client('sim3dnnqgsGgT9RmjYwqMuu+2GO1')
algo = client.algo('specrom/Google_scraper/0.1.4')
algo.set_options(timeout=300) # optional

In [24]:
query = {"query": f"soumya parekh"}
results = algo.pipe(query).result

In [27]:
type(results)

list

In [29]:
matches.scraper_response

0      -
1      -
2      -
3      -
4      -
      ..
407    -
408    -
409    -
410    -
411    -
Name: scraper_response, Length: 412, dtype: object

In [35]:
for i in range(0, 250):
    if authors.iloc[int(matches.iloc[i].author_index)].method == 'no match':
        matches.loc[i, 'scraper_response'] = 'invalid'
        print("no valid muckrack url to compare with", i)
    else:
        email_to_google = emails.iloc[int(matches.iloc[i].email_index)].email_address
        query = {"query": f"{email_to_google} muckrack"}
        results = algo.pipe(query).result
        matches.at[i, 'scraper_response'] = results

In [50]:
matches

,author_index,email_index,match_threshold,scraper_response
0,1,2080352,0.552,"[{'position': 1, 'snippet': 'Find James Fowler..."
1,2054,1764154,0.560,"[{'position': 1, 'snippet': 'newsday.com — Kev..."
2,11,205311,0.500,"[{'position': 1, 'snippet': 'Find Jenna Ciccot..."
3,2074,312243,0.615,"[{'position': 1, 'snippet': 'Find Paul Doyle o..."
4,33,38933,0.533,"[{'position': 1, 'snippet': 'Find Matt Cooper ..."
...,...,...,...,...
407,1989,6443,0.538,-
408,2011,785848,0.581,-
409,2022,809326,0.625,-
410,2023,5612,0.528,-


In [54]:
matches[102:105]

,author_index,email_index,match_threshold,scraper_response
102,2408,113467,0.615,"[{'position': 1, 'snippet': 'journalinquirer.c..."
103,2410,56631,0.591,[]
104,372,514138,0.500,[]


In [56]:
authors.iloc[2408], emails.iloc[113467], matches.iloc[102].scraper_response

(author                                  Anthony Salamone
 article_count                                         16
 request_url        https://muckrack.com/anthony-salamone
 redirect_url       https://muckrack.com/anthony-salamone
 method                                       brute force
 confidence                                             -
 first_name_df                                    anthony
 common_outlet                                  mcall.com
 author + outlet               Anthony Salamone mcall.com
 Name: 2408, dtype: object,
 email_address     anthony.armor@gmail.com
 first_name_df                     anthony
 second_name_df                      armor
 Name: 113467, dtype: object,
 [{'position': 1,
   'snippet': "journalinquirer.com — VERNON — Residents and other stakeholders will have the opportunity tonight to provide input on the town's new Plan of Conservation\xa0...",
   'title': 'Anthony Branciforte | Journal Inquirer Journalist | Muck Rack',
   'url': 'https:/

In [57]:
client = Algorithmia.client('simTpmTAYZ2FMC2MdM5b9NeODVo1')
algo = client.algo('specrom/Google_scraper/0.1.4')
algo.set_options(timeout=300) # optional

In [58]:
for i in range(103, 250):
    if authors.iloc[int(matches.iloc[i].author_index)].method == 'no match':
        matches.loc[i, 'scraper_response'] = 'invalid'
        print("no valid muckrack url to compare with", i)
    else:
        email_to_google = emails.iloc[int(matches.iloc[i].email_index)].email_address
        query = {"query": f"{email_to_google} muckrack"}
        results = algo.pipe(query).result
        matches.at[i, 'scraper_response'] = results

In [85]:
for i in range(250, 300):
    if authors.iloc[int(matches.iloc[i].author_index)].method == 'no match':
        matches.loc[i, 'scraper_response'] = 'invalid'
        print("no valid muckrack url to compare with", i)
    else:
        email_to_google = emails.iloc[int(matches.iloc[i].email_index)].email_address
        query = {"query": f"{email_to_google} muckrack"}
        results = algo.pipe(query).result
        matches.at[i, 'scraper_response'] = results

In [86]:
matches.iloc[299]

author_index                                                     1302
email_index                                                    219385
match_threshold                                                 0.621
scraper_response    [{'position': 1, 'snippet': 'Find Eduardo Medi...
Name: 299, dtype: object

In [87]:
for i in range(300):
    if matches.iloc[i].scraper_response:
        print(matches.iloc[i].scraper_response[0])
    else:
        print('empty')
    print()

{'position': 1, 'snippet': "Find James Fowler's articles, email address, contact information, Twitter and more.", 'title': 'James Fowler | Freelance Journalist | Muck Rack', 'url': 'https://muckrack.com/james-fowler-5'}

{'position': 1, 'snippet': "newsday.com — Kevin James is returning to TV next month. However, the comedian, who grew up in Stony Brook, won't be on CBS, which launched his shows\xa0...", 'title': 'David Criblez | Newsday Journalist | Muck Rack', 'url': 'https://muckrack.com/davidjcriblez'}

{'position': 1, 'snippet': "Find Jenna Ciccotelli's articles, email address, contact information, Twitter and more.", 'title': 'Jenna Ciccotelli | Boston.com, The Boston Globe ... - Muck Rack', 'url': 'https://muckrack.com/jenna-ciccotelli'}

{'position': 1, 'snippet': "Find Paul Doyle of The Guardian's articles, email address, contact information, Twitter and more.", 'title': 'Paul Doyle | The Guardian Journalist | Muck Rack', 'url': 'https://muckrack.com/paul-doyle-1'}

{'position

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

empty

{'position': 1, 'snippet': "Scott Henderson joined Furman's Education Department in 1998. Prior to that, he was a secondary social studies teacher in Chesapeake, Virginia, an English\xa0...", 'title': 'Scott Henderson – Furman University', 'url': 'https://www.furman.edu/people/scott-henderson/'}

empty

{'position': 1, 'snippet': "Find Scott Murray of The Guardian's articles, email address, contact information, Twitter and more.", 'title': 'Scott Murray | The Guardian Journalist | Muck Rack', 'url': 'https://muckrack.com/scot

In [92]:
client = Algorithmia.client('simK/0hAWWEFaereJxcXWlLvDct1')
algo = client.algo('specrom/Google_scraper/0.1.4')
algo.set_options(timeout=100) # optional

In [95]:
for i in range(300, len(matches)):
    if authors.iloc[int(matches.iloc[i].author_index)].method == 'no match':
        matches.loc[i, 'scraper_response'] = 'invalid'
        print("no valid muckrack url to compare with", i)
    else:
        email_to_google = emails.iloc[int(matches.iloc[i].email_index)].email_address
        query = {"query": f"{email_to_google} muckrack"}
        results = algo.pipe(query).result
        matches.at[i, 'scraper_response'] = results

In [96]:
for i in range(len(matches)):
    if matches.iloc[i].scraper_response:
        print(matches.iloc[i].scraper_response[0])
    else:
        print('empty')
    print()

{'position': 1, 'snippet': "Find James Fowler's articles, email address, contact information, Twitter and more.", 'title': 'James Fowler | Freelance Journalist | Muck Rack', 'url': 'https://muckrack.com/james-fowler-5'}

{'position': 1, 'snippet': "newsday.com — Kevin James is returning to TV next month. However, the comedian, who grew up in Stony Brook, won't be on CBS, which launched his shows\xa0...", 'title': 'David Criblez | Newsday Journalist | Muck Rack', 'url': 'https://muckrack.com/davidjcriblez'}

{'position': 1, 'snippet': "Find Jenna Ciccotelli's articles, email address, contact information, Twitter and more.", 'title': 'Jenna Ciccotelli | Boston.com, The Boston Globe ... - Muck Rack', 'url': 'https://muckrack.com/jenna-ciccotelli'}

{'position': 1, 'snippet': "Find Paul Doyle of The Guardian's articles, email address, contact information, Twitter and more.", 'title': 'Paul Doyle | The Guardian Journalist | Muck Rack', 'url': 'https://muckrack.com/paul-doyle-1'}

{'position

In [97]:
matches.to_pickle('../Data/google_scrape_emails.pkl')

In [105]:
def found_match(author_index, email_index, scraper_response):
    if not scraper_response:
        return 'Invalid'
    else:
        return authors.iloc[author_index].redirect_url == scraper_response[0]['url']

found_match(1, 2080352, [{'position': 1, 'snippet': "Find James Fowler's articles, email address, contact information, Twitter and more.", 'title': 'James Fowler | Freelance Journalist | Muck Rack', 'url': 'https://muckrack.com/james-fowler-5'}])

False

In [106]:
matches['email_checks_out'] = matches.apply(lambda x: found_match(x['author_index'], x['email_index'], x['scraper_response']), axis=1)

In [109]:
matches.email_checks_out.value_counts()

False      200
Invalid    123
True        89
Name: email_checks_out, dtype: int64

In [113]:
matches[matches.email_checks_out == True].match_threshold.mean()

0.6107865168539327

In [114]:
matches[matches.email_checks_out == False].match_threshold.mean()

0.5404549999999998

In [115]:
matches[matches.email_checks_out == 'Invalid'].match_threshold.mean()

0.5727804878048782